NIM  : 2702321415
Nama : Daniel

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option('display.max_columns', None)

from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
data = pd.read_csv('3B.tsv', sep = '\t')
data

,url,name,online_order,book_table,rate,votes,location,rest_type,cuisines,approx_cost(for two people),listed_in(type),sell_beverages,sell_chinese_food,sell_thai_food,sell_indian_food,sell_mediterranean_food,sell_fast_food,sell_desserts
0,https://www.zomato.com/bangalore/faasos-1-vasa...,Faasos,Yes,No,3.8,101,Vasanth Nagar,Delivery,"North Indian, Biryani, Fast Food",500,Delivery,no,no,no,yes,no,yes,no
1,https://www.zomato.com/bangalore/khanapur-expr...,Khanapur Express,No,No,NaN,0,JP Nagar,Quick Bites,"North Indian, Rajasthani",500,Delivery,no,no,no,yes,no,no,no
2,https://www.zomato.com/bangalore/russh-gastrop...,Russh Gastropub,No,No,4.2,1279,Church Street,Pub,"Continental, Italian, Burger","1,100",Drinks & nightlife,no,no,no,no,no,yes,no
3,https://www.zomato.com/bangalore/bangalir-rann...,Bangalir Rannaghar,No,No,2.9,58,Bannerghatta Road,Quick Bites,"Bengali, North Indian, Chinese",250,Dine-out,no,yes,no,yes,no,no,no
4,https://www.zomato.com/bangalore/natural-mumba...,Natural Mumbai Kulfi,Yes,No,3.5,8,BTM,Dessert Parlor,Ice Cream,130,Delivery,no,no,no,no,no,no,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,https://www.zomato.com/bangalore/sri-vasavi-re...,Sri Vasavi Restaurant,No,No,3.6,30,Bellandur,Quick Bites,"Chinese, Biryani",250,Delivery,no,yes,no,yes,no,no,no
14996,https://www.zomato.com/bangalore/hoppipola-chu...,Hoppipola,Yes,No,4.3,1708,Church Street,Lounge,"Continental, Mediterranean, European","1,300",Delivery,no,no,no,no,yes,no,no
14997,https://www.zomato.com/bangalore/ghar-ki-rasoi...,Ghar Ki Rasoi,Yes,No,3.7,50,Bellandur,Casual Dining,North Indian,500,Delivery,no,no,no,yes,no,no,no
14998,https://www.zomato.com/bangalore/ice-cubes-ind...,Ice Cubes,No,No,3.0,16,Indiranagar,Quick Bites,"Fast Food, Desserts, Beverages",300,Dine-out,yes,no,no,no,no,yes,yes


In [3]:
print(data.isnull().sum())

url                               0
name                              0
online_order                      0
book_table                        0
rate                           2264
votes                             0
location                          8
rest_type                        69
cuisines                         12
approx_cost(for two people)      92
listed_in(type)                   0
sell_beverages                   12
sell_chinese_food                12
sell_thai_food                   12
sell_indian_food                 12
sell_mediterranean_food          12
sell_fast_food                   12
sell_desserts                    12
dtype: int64


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   url                          15000 non-null  object
 1   name                         15000 non-null  object
 2   online_order                 15000 non-null  object
 3   book_table                   15000 non-null  object
 4   rate                         12736 non-null  object
 5   votes                        15000 non-null  int64 
 6   location                     14992 non-null  object
 7   rest_type                    14931 non-null  object
 8   cuisines                     14988 non-null  object
 9   approx_cost(for two people)  14908 non-null  object
 10  listed_in(type)              15000 non-null  object
 11  sell_beverages               14988 non-null  object
 12  sell_chinese_food            14988 non-null  object
 13  sell_thai_food               14

Beberapa anomali pada dataset:
- rate dan approx_cost(for two people) bertipe object
- Pada kolom rate, ditemukan nilai-nilai yang kurang tepat
- Banyaknya outlier


In [5]:
data = data.drop('url', axis = 1)

In [6]:
data['rest_type'].value_counts()

rest_type
Quick Bites                   5516
Casual Dining                 2967
Cafe                          1115
Delivery                       781
Dessert Parlor                 642
                              ... 
Casual Dining, Sweet Shop        1
Club, Casual Dining              1
Quick Bites, Mess                1
Bakery, Sweet Shop               1
Sweet Shop, Dessert Parlor       1
Name: count, Length: 85, dtype: int64

In [7]:
data['rate'].value_counts()

rate
NEW     631
3.9     593
3.7     565
3.8     552
3.7     536
       ... 
2.1       5
4.9       5
2.2       3
2.0       2
1.8       1
Name: count, Length: 63, dtype: int64

In [8]:
data['rate'].unique()

array(['3.8', nan, '4.2', '2.9', '3.5', '3.5 ', '4.4', '4.3', '4.2 ',
       '3.6', '2.6', '2.7 ', '3.1', '3.2 ', '3.6 ', '4.1 ', '3.2', '3.8 ',
       '4.5', '3.3 ', '4.4 ', '4.6', '4.1', '4.0 ', '4.5 ', '3.4', '3.1 ',
       'NEW', '3.7', '3.9 ', '3.9', '4.0', '2.4', '3.0', '3.7 ', '3.4 ',
       '2.6 ', '2.8 ', '3.3', '4.3 ', '3.0 ', '4.9 ', '2.8', '2.5 ',
       '4.6 ', '2.9 ', '2.7', '-', 'X', '4.7', '2.2 ', '4.8 ', '4.8',
       '2.1', '4.7 ', '2.3', '2.5', '2.3 ', '2.4 ', '2.1 ', '2.0 ', '4.9',
       '2.2', '1.8 '], dtype=object)

In [9]:
data['rate'] = data['rate'].replace(["NEW", "-", "X"], np.nan)
data['rate'] = data['rate'].astype(float)

In [10]:
data['rate'].isnull().values.sum()

2926

In [11]:
data['rate'].median()

3.7

In [12]:
data['rate'].fillna(3.7, inplace=True)

In [13]:
data = data.dropna()

In [14]:
data['listed_in(type)'].value_counts()

listed_in(type)
Delivery              7447
Dine-out              5161
Desserts               991
Cafes                  507
Drinks & nightlife     297
Buffet                 243
Pubs and bars          198
Name: count, dtype: int64

In [15]:
data['approx_cost(for two people)'] = data['approx_cost(for two people)'].str.replace(',', '').astype(float)


In [16]:
label_encoding = preprocessing.LabelEncoder()
data['sell_beverages'] = label_encoding.fit_transform(data['sell_beverages'])
data['sell_chinese_food'] = label_encoding.fit_transform(data['sell_chinese_food'])
data['sell_thai_food'] = label_encoding.fit_transform(data['sell_thai_food'])
data['sell_indian_food'] = label_encoding.fit_transform(data['sell_indian_food'])
data['sell_mediterranean_food'] = label_encoding.fit_transform(data['sell_mediterranean_food'])
data['sell_fast_food'] = label_encoding.fit_transform(data['sell_fast_food'])
data['sell_desserts'] = label_encoding.fit_transform(data['sell_desserts'])
data['online_order'] = label_encoding.fit_transform(data['sell_desserts'])
data['book_table'] = label_encoding.fit_transform(data['sell_desserts'])

In [17]:
ohe = OneHotEncoder()
enc = pd.DataFrame(ohe.fit_transform(data[['listed_in(type)']]).toarray(),columns=ohe.get_feature_names_out())
data = data.reset_index(drop=True)
data = pd.concat([data,enc], axis=1)
data = data.drop('listed_in(type)', axis = 1)

In [18]:
data['cuisines'].value_counts()

cuisines
North Indian                               850
North Indian, Chinese                      678
South Indian                               513
Biryani                                    246
Bakery, Desserts                           240
                                          ... 
Cafe, Continental, Beverages, Fast Food      1
Chinese, Oriya, North Indian, Bengali        1
Fast Food, Continental, Chinese              1
Cafe, Tea, Beverages, Fast Food              1
Desserts, Ice Cream, Fast Food               1
Name: count, Length: 2103, dtype: int64

In [19]:
categorical_cols = data.select_dtypes(include=['object']).columns
numeric_cols = data.select_dtypes(exclude=['object']).columns


In [20]:
outlier_counts = {}

for col in numeric_cols:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = data[(data[col] < lower_bound) | (data[col] > upper_bound)]
    
    outlier_counts[col] = len(outliers)

for col, count in outlier_counts.items():
    print(f"Number of outliers for {col}: {count}")

Number of outliers for online_order: 1795
Number of outliers for book_table: 1795
Number of outliers for rate: 856
Number of outliers for votes: 2031
Number of outliers for approx_cost(for two people): 1380
Number of outliers for sell_beverages: 2974
Number of outliers for sell_chinese_food: 0
Number of outliers for sell_thai_food: 565
Number of outliers for sell_indian_food: 0
Number of outliers for sell_mediterranean_food: 167
Number of outliers for sell_fast_food: 3057
Number of outliers for sell_desserts: 1795
Number of outliers for listed_in(type)_Buffet: 243
Number of outliers for listed_in(type)_Cafes: 507
Number of outliers for listed_in(type)_Delivery: 0
Number of outliers for listed_in(type)_Desserts: 991
Number of outliers for listed_in(type)_Dine-out: 0
Number of outliers for listed_in(type)_Drinks & nightlife: 297
Number of outliers for listed_in(type)_Pubs and bars: 198


In [21]:
data['cuisines'] = data['cuisines'].str.replace(',',' ')

In [22]:
data['cuisines'].value_counts()

cuisines
North Indian                               850
North Indian  Chinese                      678
South Indian                               513
Biryani                                    246
Bakery  Desserts                           240
                                          ... 
Cafe  Continental  Beverages  Fast Food      1
Chinese  Oriya  North Indian  Bengali        1
Fast Food  Continental  Chinese              1
Cafe  Tea  Beverages  Fast Food              1
Desserts  Ice Cream  Fast Food               1
Name: count, Length: 2103, dtype: int64

In [23]:
tf_col = data['cuisines'] + " " + data['rest_type'] + " " + data['location']
tf_col

0        North Indian  Biryani  Fast Food Delivery Vasa...
1            North Indian  Rajasthani Quick Bites JP Nagar
2           Continental  Italian  Burger Pub Church Street
3        Bengali  North Indian  Chinese Quick Bites Ban...
4                             Ice Cream Dessert Parlor BTM
                               ...                        
14839               Chinese  Biryani Quick Bites Bellandur
14840    Continental  Mediterranean  European Lounge Ch...
14841                 North Indian Casual Dining Bellandur
14842    Fast Food  Desserts  Beverages Quick Bites Ind...
14843         Desserts  Beverages Dessert Parlor Jayanagar
Length: 14844, dtype: object

In [24]:
# membuat matriks TF-IDF
tfidf = TfidfVectorizer()
tf_matrix = tfidf.fit_transform(tf_col)

In [25]:
# reset index dan hapus kolom index (mencegah index yang tidak relevan setelah pengolahan data)
data = data.reset_index()
data = data.drop('index', axis = 1)

In [26]:
scaling = StandardScaler()
num_col = ['rate', 'votes', 'approx_cost(for two people)']

for col in num_col:
    data[col] = scaling.fit_transform(data[[col]])

In [27]:
# mencari kolom dengan hanya 2 nilai unik
bin_col = [col for col in data.columns if data[col].nunique() == 2]

In [28]:
le = LabelEncoder()
for col in bin_col:
    data[col] = le.fit_transform(data[col])

In [29]:
dr = data.drop(['name','location', 'rest_type', 'cuisines'], axis = 1)

In [30]:
# mendapatkan daftar kata yang diubah menjadi fitur
tfidf_feature_names = tfidf.get_feature_names_out() 
# membuat dataframe untuk TF-IDF
tfidf_df = pd.DataFrame(tf_matrix.toarray(), columns=tfidf_feature_names)

In [31]:
# menggabungkan dataframe dr dan TF-IDF ke dalam satu dataframe baru
df_com = pd.concat([dr.reset_index(drop=True), tfidf_df.reset_index(drop=True)], axis=1)
df_com.head()

,online_order,book_table,rate,votes,approx_cost(for two people),sell_beverages,sell_chinese_food,sell_thai_food,sell_indian_food,sell_mediterranean_food,sell_fast_food,sell_desserts,listed_in(type)_Buffet,listed_in(type)_Cafes,listed_in(type)_Delivery,listed_in(type)_Desserts,listed_in(type)_Dine-out,listed_in(type)_Drinks & nightlife,listed_in(type)_Pubs and bars,1st,2nd,3rd,4th,5th,6th,7th,8th,afghan,afghani,african,airport,american,andhra,arabian,asian,assamese,australian,awadhi,bakery,banashankari,banaswadi,bangalore,bannerghatta,bar,basavanagudi,basaveshwara,bbq,bel,belgian,bellandur,bengali,beverage,beverages,bhima,bihari,biryani,bites,block,bohri,bommanahalli,brigade,brookefield,btm,bubble,burger,burmese,cafe,cafee,cantonese,casual,central,charcoal,chettinad,chicken,chinese,church,city,club,coffee,commercial,confectionery,continental,course,court,cream,cunningham,cv,delivery,dessert,desserts,dhaba,dining,dogs,domlur,drinks,east,eastern,ejipura,electronic,european,fast,fine,finger,food,frazer,french,garden,german,goan,greek,grill,gujarati,hbr,healthy,hebbal,hennur,hosur,hot,hsr,hyderabadi,ice,indian,indiranagar,indonesian,infantry,irani,iranian,italian,itpl,jalahalli,japanese,jayanagar,jeevan,jewish,jp,juices,kaggadasapura,kalyan,kammanahalli,kanakapura,kashmiri,kebab,kengeri,kerala,kiosk,konkan,koramangala,korean,kr,kumaraswamy,langford,lankan,lavelle,layout,lebanese,lounge,lucknowi,madras,magadi,maharashtrian,main,majestic,malaysian,malleshwaram,mangalorean,marathahalli,market,marks,mediterranean,mess,mex,mexican,mg,microbrewery,middle,mithai,modern,momos,mongolian,mughlai,mysore,naga,nagar,nagarbhavi,nagawara,nepalese,new,north,old,only,oriya,paan,pan,parlor,parsi,pizza,portuguese,pub,puram,quick,race,rajajinagar,rajarajeshwari,rajasthani,raman,rammurthy,residency,richmond,road,roast,rolls,rt,russian,sadashiv,sahakara,salad,sandwich,sanjay,sankey,sarjapur,seafood,seshadripuram,shanti,shivajinagar,shop,sindhi,singaporean,south,spanish,sri,st,steak,street,sushi,sweet,takeaway,tea,tex,thai,thippasandra,tibetan,town,truck,turkish,ulsoor,uttarahalli,varthur,vasanth,vegan,vietnamese,vijay,west,whitefield,wilson,wraps,yelahanka,yeshwantpur
0,0,0,0.249406,-0.227935,-0.126246,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.321148,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.353472,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.297406,0.0,0.0,0.248559,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.179812,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.355769,0.0,0.0,0.0,0.0,0.197654,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.650936,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,-0.000826,-0.351732,-0.126246,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.208974,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.186919,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.45062,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [32]:
df_com.shape

(14844, 257)

In [33]:
n_components = 200  
# reduksi dimensi menjadi untuk memudahkan pengolahan data.
svd = TruncatedSVD(n_components=n_components)
features_reduced = svd.fit_transform(df_com)

features_reduced

array([[-2.92620268e-01,  1.31994507e+00, -4.14687438e-01, ...,
         1.14944628e-04,  5.80807876e-04, -6.02369930e-03],
       [-4.88053588e-01,  1.13201341e+00, -1.46706875e-01, ...,
         1.48903575e-04, -3.96340330e-04,  1.49521242e-03],
       [ 2.12719999e+00,  5.01378850e-01, -1.21240399e-01, ...,
        -5.14853043e-03,  2.35459614e-05,  1.35962087e-03],
       ...,
       [-4.22965608e-01,  1.16876905e+00, -1.06556517e-01, ...,
        -1.04002756e-03,  1.01706577e-04,  1.10285062e-03],
       [-1.59533928e+00,  7.66883218e-01, -2.33339371e-01, ...,
        -1.58338832e-04,  3.97252740e-04, -7.08473481e-04],
       [-5.19914277e-02,  7.89874786e-01, -1.85142639e+00, ...,
        -2.17903673e-03, -4.21718286e-05, -5.89084399e-04]])

In [39]:
def recommendations_with_name(item_name, df, features_matrix, top_n=5):


    if item_name in data['name'].values:
        item_idx = data[data['name'] == item_name].index[0]
    else:
        return f"Item dengan nama {item_name} tidak ditemukan dalam rekomendasi."

    item_features = features_matrix[item_idx]


    similarity_scores = cosine_similarity([item_features], features_matrix)[0]

    top_indices = np.argsort(similarity_scores)[::-1][1:top_n+1]  
    recommendations = data.iloc[top_indices][['name', 'location', 'cuisines', 'rate']]
    return recommendations

In [40]:
recommendations = recommendations_with_name('Burger King', data, features_reduced)
recommendations


,name,location,cuisines,rate
11271,Burger King,Kammanahalli,Burger Fast Food Beverages,0.499639
14313,Burger King,New BEL Road,Burger Fast Food Beverages,0.749872
5893,Burger King,Commercial Street,Burger Fast Food Beverages,0.499639
9949,Burger King,Electronic City,Burger Fast Food Beverages,0.249406
8746,Burger King,Rajajinagar,Burger Fast Food Beverages,0.499639


In [41]:
recommendations = recommendations_with_name("Cafe Coffee Day", data, features_reduced)
recommendations


,name,location,cuisines,rate
35,Cafe Coffee Day,Jayanagar,Cafe Fast Food,-0.501291
13780,Cafe Coffee Day,Wilson Garden,Cafe Fast Food,-0.000826
2150,Cafe Coffee Day,Sarjapur Road,Cafe Fast Food,-1.502221
1695,Cafe Arabica,Bannerghatta Road,Cafe Bakery Arabian Fast Food,0.249406
691,Art Cafe,Malleshwaram,Cafe,-0.501291


In [42]:
recommendations = recommendations_with_name('Krispy Kreme', data, features_reduced)
recommendations

,name,location,cuisines,rate
8056,Krispy Kreme,HSR,Desserts Beverages,0.499639
9345,Waffle Stories,HSR,Desserts Beverages,-0.000826
14710,Waffle Stories,HSR,Desserts Beverages,-0.000826
8602,Waffle Stories,HSR,Desserts Beverages,-0.000826
12963,Bubble Bee,HSR,Beverages Bubble Tea Desserts,0.499639


Dari hasil evaluasi sistem rekomendasi ini, dapat dirincikan sebagai berikut:
- Burger King -> hasil rekomendasi masih berkaitan dengan input (Burger King). Jika dilihat, sistem merekomendasikan cabang dari nama restoran yang diinput.

- Cafe Coffee Day -> mirip dengan Burger King, dimana sistem merekomendasikan cabang dari restoran tersebut. Selain itu, sistem juga merekomendasikan melalui kesamaan cuisines yaitu Cafe.

- Krispy Kreme -> Sistem merekomendasikan 1 restoran yang sama dengan input. Lalu ada rekomendasi yang memiliki kesamaan dari sisi lokasi dan cuisines.

Sistem mampu memberikan rekomendasi yang relevan berdasarkan nama restoran yang diinput, serta kesamaan dalam hal lokasi dan jenis makanan (cuisines). Sistem menunjukkan performanya dalam memberikan rekomendasi cabang restoran yang sama atau yang serupa, sesuai dengan input.